# 2 start pods on Azure AKS

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## login AKS

In [ ]:
az login --tenant ${TENANT}

## create wildcard TLS cert files using Let's Encrypt

### create tls cert files using let's encrypt

_Outside of this notebook_
1. open a ternminal.
1. run a command displayed below.

In [ ]:
echo "docker run -it -v ${PJ_ROOT}/secrets:/etc/letsencrypt certbot/certbot certonly --manual --domain *.${DOMAIN} --email ${EMAIL} --no-eff-email --agree-tos --manual-public-ip-logging-ok --preferred-challenges dns-01 --server https://acme-v02.api.letsencrypt.org/directory"

_Outside of this notebook_
1. wait until to add a DNS TXT Record like below.

notice) **replace XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX with the 'DNS TXT record' value displayed in opened terminal.**

In [ ]:
export DNS_TXT="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
az network dns record-set txt add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge" --value "${DNS_TXT}"

_Outside of this notebook_
1. in order to continue creating cert file process, press enter key in the opened terminal.
1. confirm to be displayed the below message at the opened terminal.
```
Congratulations! Your certificate and chain have been saved at:
```

In [ ]:
az network dns record-set txt remove-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "_acme-challenge" --value "${DNS_TXT}"

## start RabbitMQ cluster on AKS
[rabbitmq](https://www.rabbitmq.com/)

### register cert files to kubernetes secrets

In [ ]:
kubectl create secret generic rabbitmq-certifications --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/fullchain.pem --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/cert.pem --from-file=${PJ_ROOT}/secrets/live/${DOMAIN}/privkey.pem

### start RabbitMQ

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-rbac.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-azure-services.yaml

In [ ]:
kubectl apply -f rabbitmq/rabbitmq-azure-statefulset.yaml

In [ ]:
kubectl get pods -l app=rabbitmq

example)

```
NAME         READY     STATUS    RESTARTS   AGE
rabbitmq-0   1/1       Running   0          6m
rabbitmq-1   1/1       Running   0          5m
rabbitmq-2   1/1       Running   0          4m
```

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl cluster_status

example)
```
Cluster status of node rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local ...
[{nodes,[{disc,['rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local']}]},
 {running_nodes,['rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',
                 'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local']},
 {cluster_name,<<"rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local">>},
 {partitions,[]},
 {alarms,[{'rabbit@rabbitmq-2.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-1.rabbitmq.default.svc.cluster.local',[]},
          {'rabbit@rabbitmq-0.rabbitmq.default.svc.cluster.local',[]}]}]
```

### change password of `guest`

#### for macOS

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl change_password guest $(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)

#### for Ubuntu

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl change_password guest $(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32)

### register users to RabbitMQ

In [ ]:
for e in $(env); do
  if [[ "${e}" =~ ^MQTT__([[:alnum:]_-]+)=([[:alnum:]_-]+)$ ]]; then
    username=${BASH_REMATCH[1]}
    password=${BASH_REMATCH[2]}
    
    kubectl exec rabbitmq-0 -- rabbitmqctl add_user ${username} ${password}
    kubectl exec rabbitmq-0 -- rabbitmqctl set_permissions -p / ${username} ".*" ".*" ".*"
  fi
done

In [ ]:
kubectl exec rabbitmq-0 -- rabbitmqctl list_users

example)
```
Listing users ...
guest	[administrator]
iotagent	[]
```

### register DNS A Record for RabbitMQ

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l app=rabbitmq -l service=mqtts

expected)
```
NAME             TYPE           CLUSTER-IP   EXTERNAL-IP   PORT(S)          AGE
rabbitmq-mqtts   LoadBalancer   10.0.96.7    XX.XX.XX.XX   8883:30299/TCP   21m
```

In [ ]:
MQTTS_IPADDR=$(kubectl get services -l app=rabbitmq -l service=mqtts -o jsonpath='{.items[0].status.loadBalancer.ingress[0].ip}')
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "mqtt" --ipv4-address "${MQTTS_IPADDR}"

In [ ]:
nslookup mqtt.${DOMAIN}

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${PJ_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /test -m "test"

example)
```
Client mosqpub|52150-Nobuyukin sending CONNECT
Client mosqpub|52150-Nobuyukin received CONNACK
Client mosqpub|52150-Nobuyukin sending PUBLISH (d0, q0, r0, m1, '/test', ... (4 bytes))
Client mosqpub|52150-Nobuyukin sending DISCONNECT
```

## start mondodb cluster on AKS
[mongodb](https://www.mongodb.com/)

create 'mongodb' takes a few minutes.

In [ ]:
helm install --name mongodb -f mongodb/mongodb-replicaset-values-azure.yaml stable/mongodb-replicaset

In [ ]:
kubectl get PersistentVolumeClaims -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME                              STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS      AGE
mongodb-storage-claim-mongodb-0   Bound     pvc-386ac3ba-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   5m
mongodb-storage-claim-mongodb-1   Bound     pvc-88455199-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   3m
mongodb-storage-claim-mongodb-2   Bound     pvc-b74ec32c-856e-11e8-a3b4-026b849f8c40   30Gi       RWO            managed-premium   2m
```

In [ ]:
kubectl get statefulsets -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME      READY   AGE
mongodb   3/3     5m32s
```

In [ ]:
kubectl get pods -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME        READY   STATUS    RESTARTS   AGE
mongodb-0   1/1     Running   0          5m55s
mongodb-1   1/1     Running   0          4m
mongodb-2   1/1     Running   0          2m27s
```

In [ ]:
kubectl get services -l release=mongodb -l app=mongodb-replicaset

example)
```
NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)     AGE
mongodb          ClusterIP   None         <none>        27017/TCP   6m15s
mongodb-client   ClusterIP   None         <none>        27017/TCP   6m15s
```

In [ ]:
kubectl exec mongodb-0 -c mongodb-replicaset -- mongo --eval 'printjson(rs.status().members.map(function(e) {return {name: e.name, stateStr:e.stateStr};}))'

expected)
```
MongoDB shell version v4.1.10
connecting to: mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("dcf9bc7d-3d4b-4f57-af18-22f1d48dd923") }
MongoDB server version: 4.1.10
[
	{
		"name" : "mongodb-0.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "PRIMARY"
	},
	{
		"name" : "mongodb-1.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	},
	{
		"name" : "mongodb-2.mongodb.default.svc.cluster.local:27017",
		"stateStr" : "SECONDARY"
	}
]
```

## start ambassador on AKS

[ambassador](https://www.getambassador.io/)

### register cert files of ambassador to kubernetes secrets

In [ ]:
kubectl create secret tls ambassador-certs --cert=${PJ_ROOT}/secrets/live/${DOMAIN}/fullchain.pem --key=${PJ_ROOT}/secrets/live/${DOMAIN}/privkey.pem

### start ambassador

In [ ]:
kubectl apply -f ambassador/ambassador-azure-services.yaml

In [ ]:
kubectl apply -f ambassador/ambassador-deployment.yaml

In [ ]:
kubectl get pods -l app=ambassador

example)
```
NAME                         READY     STATUS    RESTARTS   AGE
ambassador-cf99ff6bb-45jpl   2/2       Running   0          39s
ambassador-cf99ff6bb-8kht7   2/2       Running   0          39s
ambassador-cf99ff6bb-lv2tk   2/2       Running   0          39s
```

**confirm that "EXTERNAL-IP" is assigned**

In [ ]:
kubectl get services -l app=ambassador

example)
```
NAME         TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)                      AGE
ambassador   LoadBalancer   10.0.191.59   YY.YY.YY.YY   443:30357/TCP,80:32755/TCP   4m
```

### register DNS A Record for ambassador

In [ ]:
HTTPS_IPADDR=$(kubectl get services -l app=ambassador -o json | jq '.items[0].status.loadBalancer.ingress[0].ip' -r)
az network dns record-set a add-record --resource-group ${DNS_ZONE_RG} --zone-name "${DOMAIN}" --record-set-name "api" --ipv4-address "${HTTPS_IPADDR}"

In [ ]:
nslookup api.${DOMAIN}

In [ ]:
curl -i https://api.${DOMAIN}

expected)
```
HTTP/1.1 404 Not Found
date: Fri, 25 May 2018 00:47:41 GMT
server: envoy
content-length: 0
```

## start authorization & authentication service on AKS

### create `secrets/auth-tokens.json`

example)
```json
[
  {
    "host": "api\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "nrWtb8sS0MmwlkhHXv0DC6orPMpFFbni",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
        },
        {
          "token": "9THT3Ebr2gDOpZYNk036v7qjQXw8j9N3",
          "allowed_paths": ["^/visualizer/positions/$"]
        }
      ],
      "basic_auths": [
        {
          "username": "user1",
          "password": "Y5vEWHLAa5Y4vjqP",
          "allowed_paths": ["/controller/web/"]
        },
        {
          "username": "visualizer",
          "password": "g2Kygzj6fmf9gh6T",
          "allowed_paths": ["/visualizer/locus/"]
        }
      ],
      "no_auths": {
        "allowed_paths": ["^.*/static/.*$"]
      }
    }
  },
  {
    "host": "kibana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "yW7FvSGD",
          "password": "6BoTFE5xfUlX3ssV",
          "allowed_paths": ["^.*$"]
        }
      ],
      "no_auths": {
        "allowed_paths": []
      }
    }
  },
  {
    "host": "grafana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": ["^.*$"]
      }
    }
  }
]
```

#### for macOS

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": "api\\\\..+$",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
                }, {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/visualizer/positions/$"]
                }
            ],
            "basic_auths": [
                {
                    "username": "user1",
                    "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["/controller/web/"]
                }, {
                    "username": "visualizer",
                    "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["/visualizer/locus/"]
                }
            ],
            "no_auths": {
                "allowed_paths": ["^.*/static/.*$"]
            }
        }
    }, {
        "host": "kibana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [
                {
                    "username": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 8)",
                    "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["^.*$"]
                }
            ],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }, {
        "host": "grafana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": ["^.*$"]
            }
        }
    }
]
__EOS__

#### for Ubuntu

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": "api\\\\..+$",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$"]
                }, {
                    "token": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/visualizer/positions/$"]
                }
            ],
            "basic_auths": [
                {
                    "username": "user1",
                    "password": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["/controller/web/"]
                }, {
                    "username": "visualizer",
                    "password": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["/visualizer/locus/"]
                }
            ],
            "no_auths": {
                "allowed_paths": ["^.*/static/.*$"]
            }
        }
    }, {
        "host": "kibana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [
                {
                    "username": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 8)",
                    "password": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["^.*$"]
                }
            ],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }, {
        "host": "grafana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": ["^.*$"]
            }
        }
    }
]
__EOS__

### register auth-tokens to kubernetes secrets

In [ ]:
kubectl create secret generic auth-tokens --from-file=./secrets/auth-tokens.json

### start auth service

In [ ]:
kubectl apply -f auth/auth-service.yaml

In [ ]:
kubectl apply -f auth/auth-deployment.yaml

In [ ]:
kubectl get pods -l app=auth

example)
```
NAME                    READY   STATUS    RESTARTS   AGE
auth-67ff6bd94b-bkk96   1/1     Running   0          23s
auth-67ff6bd94b-bqstl   1/1     Running   0          23s
auth-67ff6bd94b-p2xzt   1/1     Running   0          23s
```

In [ ]:
kubectl get services -l app=auth

example)
```
NAME   TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
auth   ClusterIP   10.0.221.96   <none>        3000/TCP   43s
```

## start fiware orion on AKS
[fiware orion](https://catalogue-server.fiware.org/enablers/publishsubscribe-context-broker-orion-context-broker)

In [ ]:
kubectl apply -f orion/orion-service.yaml

In [ ]:
kubectl apply -f orion/orion-deployment.yaml

In [ ]:
kubectl get pods -l app=orion

example)
```
NAME                     READY     STATUS    RESTARTS   AGE
orion-54f5cdcb5d-d2pt5   1/1       Running   0          56s
orion-54f5cdcb5d-hv274   1/1       Running   0          56s
orion-54f5cdcb5d-xbnx2   1/1       Running   0          56s
```

In [ ]:
kubectl get services -l app=orion

example)
```
NAME      TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
orion     ClusterIP   10.0.44.126   <none>        1026/TCP   1m
```

In [ ]:
curl -i https://api.${DOMAIN}/orion/v2/entities/

example)
```
HTTP/1.1 401 Unauthorized
content-length: 60
content-type: text/plain
www-authenticate: Bearer realm="token_required"
date: Thu, 14 Feb 2019 06:32:37 GMT
server: envoy

{"authorized":false,"error":"missing Header: authorization"}
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" https://api.${DOMAIN}/orion/v2/entities/

example)
```
HTTP/1.1 200 OK
content-length: 2
content-type: application/json
fiware-correlator: 4731eb48-4dc1-11e8-b1a2-0a580af4010a
date: Wed, 02 May 2018 04:28:35 GMT
x-envoy-upstream-service-time: 5
server: envoy

[]
```

**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware idas(iotagent-ul) on AKS
[fiware IDAS(iotagent-ul)](https://catalogue-server.fiware.org/enablers/backend-device-management-idas)

### start idas

In [ ]:
env IOTA_PASSWORD=${MQTT__iotagent} envsubst < idas/rb-config.js > /tmp/rb-config.js
kubectl create secret generic iotagent-config --from-file /tmp/rb-config.js
rm /tmp/rb-config.js

In [ ]:
kubectl apply -f idas/iotagent-ul-service.yaml

In [ ]:
kubectl apply -f idas/iotagent-ul-deployment.yaml

In [ ]:
kubectl get pods -l app=iotagent-ul

example)
```
NAME                           READY     STATUS    RESTARTS   AGE
iotagent-ul-79685b64bf-8krps   1/1       Running   0          3m
iotagent-ul-79685b64bf-m6nlg   1/1       Running   0          3m
iotagent-ul-79685b64bf-mjpbl   1/1       Running   0          3m
```

In [ ]:
kubectl get services -l app=iotagent-ul

example)
```
NAME          TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)             AGE
iotagent-ul   ClusterIP   10.0.180.155   <none>        4041/TCP,7896/TCP   43s
```

In [ ]:
TOKEN=$(cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: test" -H "Fiware-Servicepath: /*" https://api.${DOMAIN}/idas/ul20/manage/iot/services/

example)
```
HTTP/1.1 200 OK
x-powered-by: Express
fiware-correlator: c114fc5e-b4a2-40f6-b7fe-1d68369784e5
content-type: application/json; charset=utf-8
content-length: 25
etag: W/"19-WMYe0U6ocKhQjp+oaVnMHLdbylc"
date: Wed, 02 May 2018 06:16:18 GMT
x-envoy-upstream-service-time: 9
server: envoy

{"count":0,"services":[]}
```

**If you can't get 200 OK, please restart all ambassador's pods like "ambassador-cf99ff6bb-45jpl", "ambassador-cf99ff6bb-8kht7" and "ambassador-cf99ff6bb-lv2tk"**

## start fiware cygnus (mongodb sink) on AKS
[fiware cygnus](https://catalogue-server.fiware.org/enablers/cygnus)

### start cygnus-mongo

In [ ]:
kubectl apply -f cygnus/cygnus-mongo-service.yaml

In [ ]:
kubectl apply -f cygnus/cygnus-mongo-deployment.yaml

In [ ]:
kubectl get pods -l app=cygnus-mongo

example)
```
NAME                            READY     STATUS    RESTARTS   AGE
cygnus-mongo-588cb8cb46-42bcn   1/1       Running   0          1m
cygnus-mongo-588cb8cb46-k9cr2   1/1       Running   0          1m
cygnus-mongo-588cb8cb46-phvst   1/1       Running   0          1m
```

In [ ]:
kubectl get services -l app=cygnus-mongo

example)
```
NAME           TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)             AGE
cygnus-mongo   ClusterIP   10.0.10.63   <none>        5050/TCP,8081/TCP   1m
```